In [ ]:
import keras

# Data preparation

In [ ]:
dataset_path = 'dataset'

train_path = dataset_path + '/train'
train_images_path = train_path + '/images'
train_annotations_path = train_path + '/annotations'

valid_path = dataset_path + '/valid'
valid_images_path = valid_path + '/images'
valid_annotations_path = valid_path + '/annotations'

test_path = dataset_path + '/test'
test_images_path = test_path + '/images'
test_annotations_path = test_path + '/annotations'

In [ ]:
# images are .png images
# annotations are .json files
# and have the following format:
"""
{
    "version": "5.4.1",
    "flags": {},
    "shapes": [
        {
            "label": "femurR",
            "points": [
                [
                    598.125,
                    892.5
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurR",
            "points": [
                [
                    658.125,
                    903.125
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurR",
            "points": [
                [
                    670.625,
                    983.75
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurL",
            "points": [
                [
                    1170.0,
                    892.5
                ]
            ],
            "group_id": null,
            "description": null,
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurL",
            "points": [
                [
                    1115.0,
                    894.375
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurL",
            "points": [
                [
                    1086.25,
                    969.375
                ]
            ],
            "group_id": null,
            "description": null,
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "acetabulumR",
            "points": [
                [
                    611.25,
                    871.25
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "acetabulumL",
            "points": [
                [
                    1148.75,
                    858.75
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        }
    ],
    "imageHeight": 2140,
    "imageWidth": 1760
}
"""
# the model should be able to receive the image and predict the points for all the labels

In [ ]:
# here we put the valid images in the train directory
# because we are doing the split during the training with validation_split = 0.2

import shutil
import os

# Define the source and destination directories
valid_images_path = os.path.join(dataset_path, 'valid', 'images')
train_images_path = os.path.join(dataset_path, 'train', 'images')

# Get a list of all the image files in the valid directory
valid_image_files = os.listdir(valid_images_path)

# Move each file to the train directory
for file_name in valid_image_files:
    shutil.move(os.path.join(valid_images_path, file_name), train_images_path)

In [ ]:
import os
import json
import cv2
import numpy as np
from glob import glob
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the splits
splits = ['train', 'valid', 'test']

# Function to sort points
def sort_points(annotation):
    # Define the order of labels
    label_order = ['femurR', 'femurL', 'acetabulumR', 'acetabulumL']

    # Create a list of tuples, each containing a point and its corresponding label
    points_with_labels = [(point, shape['label']) for shape in annotation['shapes'] for point in shape['points']]
    # Sort the points based on the predefined label order
    points_with_labels.sort(key=lambda x: label_order.index(x[1]))
    # Remove the labels, leaving only the sorted points
    sorted_points = [point for point, label in points_with_labels]

    return sorted_points

# Function to load images and annotations
def load_data(images_path, annotations_path):
    # Get a list of all image files and annotation files
    image_files = sorted(glob(os.path.join(images_path, '*.png')))
    annotation_files = sorted(glob(os.path.join(annotations_path, '*.json')))

    #print(f'Found {len(image_files)} image files and {len(annotation_files)} annotation files.')

    images = []
    annotations = []
    image_paths = []

    for image_file, annotation_file in zip(image_files, annotation_files):
        # Load and resize the image
        image = load_img(image_file, target_size=(64, 64))
        image = img_to_array(image)
        images.append(image)

        # Load and process the annotation
        with open(annotation_file, 'r') as f:
            annotation = json.load(f)
        points = sort_points(annotation)
        annotations.append(points)

        # Save the image path
        image_paths.append(image_file)

    return np.array(images), np.array(annotations), image_paths

# Load data for each split
for split in splits:
    images_path = os.path.join(dataset_path, split, 'images')
    annotations_path = os.path.join(dataset_path, split, 'annotations')
    split_images, split_annotations, split_image_paths = load_data(images_path, annotations_path)

    if split == 'train':
        train_images, train_annotations, train_image_paths = split_images, split_annotations, split_image_paths
    elif split == 'valid':
        valid_images, valid_annotations, valid_image_paths = split_images, split_annotations, split_image_paths
    elif split == 'test':
        test_images, test_annotations, test_image_paths = split_images, split_annotations, split_image_paths

    print(f'Loaded {len(split_images)} images and {len(split_annotations)} annotations for {split}.')

# Model load

In [ ]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load the pre-trained ResNet50V2 model, excluding the top layer
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Determine the number of points in the annotations
# You can calculate it automatically based on the shape of your training annotations
# num_points = train_annotations.shape[1]
num_points = 16

# Add the output layer
predictions = Dense(num_points)(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Define the optimizer with a specific learning rate
optimizer = Adam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Print the model summary
model.summary()


In [ ]:
# Just to check how the model is receving all the data

# Convert the lists to numpy arrays
train_images = np.array(train_images)
train_annotations = np.array(train_annotations)

# Print the first image and annotation before processing
print("Before processing:")
print("First image path:", train_image_paths[0])
print("First image shape:", train_images[0].shape)
print("First image data:", train_images[0])
print("First annotation shape:", train_annotations[0].shape)
print("First annotation data:", train_annotations[0])

# Reshape the annotations and normalize the images
train_annotations = train_annotations.reshape(len(train_annotations), -1)
train_images = train_images / 255.0

# Print the first image and annotation after processing
print("\nAfter processing:")
print("First image path:", train_image_paths[0])
print("First image shape:", train_images[0].shape)
print("First image data:", train_images[0])
print("First annotation shape:", train_annotations[0].shape)
print("First annotation data:", train_annotations[0])

# Training, saving and plot

In [ ]:
# Convert the lists to numpy arrays
train_images = np.array(train_images)
train_annotations = np.array(train_annotations).reshape(len(train_annotations), -1)

# Normalize the images
train_images = train_images / 255.0

# Train the model
print(f'Training on {len(train_images)} images')
history = model.fit(train_images, train_annotations, epochs=30, validation_split=0.2, batch_size=32)

In [ ]:
model.save('saved_model/model.h5')

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Plot training & validation MAE values
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['mae'])
    plt.plot(history.history['val_mae'])
    plt.title('Model MAE')
    plt.ylabel('MAE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

# After training the model, plot the history
plot_training_history(history)

# Test

In [ ]:
# TODO: Test the model on the test images

# Prediction

In [ ]:
from keras.preprocessing import image
import numpy as np

# Load the image
img = image.load_img('dataset/test/images/1.2.276.0.74.3.362140177.2017410.115458.1.1_0001_000002_151393206006f8.png', target_size=(64, 64))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Add an extra dimension because the model expects batches of images
img_batch = np.expand_dims(img_array, axis=0)

# Normalize the image
img_preprocessed = img_batch / 255.0

# Make the prediction
prediction = model.predict(img_preprocessed)

print(prediction)

# TODO: Need to assign the points to the labels
# label_order = ['femurR', 'femurL', 'acetabulumR', 'acetabulumL']

# Model load

In [ ]:
# TODO: Future work, having some troubles with this part
